<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/01_Mask2Former_Recopilacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Evaluador Mask2Former para recolección de Métricas
**Recopilación de todas las métricas posibles sin ground truth.**

- Autor: Jesús L.
- Proyecto: TFM. Evaluación comparativa de técnicas de segmentación.

In [36]:
import torch
import numpy as np
import cv2
import json
import time
import psutil
import os
import gc
from datetime import datetime
from pathlib import Path
import hashlib

from transformers import AutoImageProcessor, AutoModelForUniversalSegmentation, AutoModelForSemanticSegmentation
from PIL import Image, ImageStat
import matplotlib.pyplot as plt
from skimage import measure
from tqdm import tqdm

In [37]:
# ==========================================
# 1. Montar Google Drive
# ==========================================
from google.colab import drive
drive.mount('/content/drive')

# Ruta a tu dataset en Google Drive
DATASET_PATH = "/content/drive/MyDrive/TFM/mask2former/imagenes"
# Carpeta donde se guardarán los resultados
OUTPUT_PATH = "/content/drive/MyDrive/TFM/mask2former/resultados"

# Crear carpeta de resultados si no existe
os.makedirs(OUTPUT_PATH, exist_ok=True)

# Lista de modelos a evaluar
MODELOS = [
    "facebook/mask2former-swin-large-coco-instance",
    "facebook/maskformer-swin-base-coco",
    "facebook/mask2former-swin-base-ade-semantic"
]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
class RecolectorMetricasCompletas:
    """
    Recolecta todas las métricas posibles de una segmentación sin ground truth.
    """
    def __init__(self, config):
        self.config = config

        # Detección automática de arquitectura
        model_name = config['model_name'].lower()
        if "mask2former" in model_name:
            self.processor = AutoImageProcessor.from_pretrained(config['model_name'])
            self.model = AutoModelForUniversalSegmentation.from_pretrained(config['model_name'])
        elif "maskformer" in model_name:
            self.processor = AutoImageProcessor.from_pretrained(config['model_name'])
            self.model = AutoModelForSemanticSegmentation.from_pretrained(config['model_name'])
        else:
            raise ValueError(f"Arquitectura no reconocida para {config['model_name']}")

        self.device = torch.device(config['device'])
        self.model.to(self.device)
        self.model.eval()

        # Crear directorios
        self.output_dir = Path(config['output_dir'])
        self.output_dir.mkdir(exist_ok=True)
        (self.output_dir / 'datos_completos').mkdir(exist_ok=True)

        self.resultados = []
        print(f"Modelo cargado en {self.device}")


    def preprocesar_imagen(ruta, max_size=1024):
      img = Image.open(ruta).convert("RGB")
      img.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
      return img

    def procesar_imagen(self, ruta_imagen):
        """Procesa una imagen y recolecta TODAS las métricas posibles."""

        inicio = time.time()

        try:
            # 1. CARGA Y ANÁLISIS BÁSICO DE IMAGEN
            imagen_pil = self.preprocesar_imagen(ruta_imagen)
            imagen_np = np.array(imagen_pil)
            h, w = imagen_np.shape[:2]
            h, w = imagen_np.shape[:2]

            # Hash único para la imagen
            ruta_imagen = Path(ruta_imagen)
            hash_img = hashlib.md5(open(ruta_imagen, 'rb').read()).hexdigest()[:12]

            # Estadísticas básicas de la imagen
            stat = ImageStat.Stat(imagen_pil)
            gray = cv2.cvtColor(imagen_np, cv2.COLOR_RGB2GRAY)

            # 2. ANÁLISIS VISUAL DE LA IMAGEN (sin modelo)
            metricas_imagen = {
                # Básicas
                'archivo': os.path.basename(ruta_imagen),
                'hash': hash_img,
                'resolucion_w': w,
                'resolucion_h': h,
                'aspect_ratio': w/h,
                'area_total': w*h,

                # Color y brillo
                'brillo_promedio': float(np.mean(gray)),
                'brillo_std': float(np.std(gray)),
                'rgb_mean_r': float(stat.mean[0]),
                'rgb_mean_g': float(stat.mean[1]),
                'rgb_mean_b': float(stat.mean[2]),
                'rgb_std_r': float(stat.stddev[0]),
                'rgb_std_g': float(stat.stddev[1]),
                'rgb_std_b': float(stat.stddev[2]),

                # Textura y complejidad
                'varianza_laplacian': float(cv2.Laplacian(gray, cv2.CV_64F).var()),
                'entropia': self._calcular_entropia(gray),
                'densidad_bordes': self._calcular_densidad_bordes(gray),

                # HSV
                'saturacion_media': float(np.mean(cv2.cvtColor(imagen_np, cv2.COLOR_RGB2HSV)[:,:,1])),
                'saturacion_std': float(np.std(cv2.cvtColor(imagen_np, cv2.COLOR_RGB2HSV)[:,:,1])),

                # Contraste local
                'contraste_local_std': float(np.std([cv2.Laplacian(gray[i:i+32, j:j+32], cv2.CV_64F).var()
                                                   for i in range(0, h-32, 32)
                                                   for j in range(0, w-32, 32) if i+32<h and j+32<w]))
            }

            # 3. RECURSOS ANTES DE INFERENCIA
            recursos_antes = self._obtener_recursos()

            # 4. INFERENCIA DEL MODELO
            inputs = self.processor(images=imagen_pil, return_tensors="pt").to(self.device)

            tiempo_inferencia_inicio = time.time()
            with torch.no_grad():
                outputs = self.model(**inputs)

            tiempo_inferencia = (time.time() - tiempo_inferencia_inicio) * 1000

            # 5. POST-PROCESAMIENTO CON MÚLTIPLES UMBRALES
            umbrales = [0.1, 0.3, 0.5, 0.7, 0.9]
            resultados_por_umbral = {}

            for umbral in umbrales:
                resultado = self.processor.post_process_instance_segmentation(
                    outputs, target_sizes=[(h, w)], threshold=umbral
                )[0]

                # Extraer personas (clase 0)
                mascaras_personas = []
                scores_personas = []

                if 'labels' in resultado and 'masks' in resultado:
                    for i, (label, score) in enumerate(zip(resultado["labels"], resultado["scores"])):
                        if label == 0:  # persona
                            mask = resultado["masks"][i].cpu().numpy().squeeze() > 0.5
                            mascaras_personas.append(mask)
                            scores_personas.append(float(score.item()))

                # Máscara combinada de todas las personas
                if mascaras_personas:
                    mascara_combinada = np.logical_or.reduce(mascaras_personas)
                else:
                    mascara_combinada = np.zeros((h, w), dtype=bool)

                # Métricas por umbral
                resultados_por_umbral[f'umbral_{umbral}'] = {
                    'num_detecciones': len(mascaras_personas),
                    'scores': scores_personas,
                    'area_segmentada': int(np.sum(mascara_combinada)),
                    'porcentaje_imagen': float(np.sum(mascara_combinada) / (h*w) * 100),
                    'coherencia': self._metricas_coherencia(mascara_combinada)
                }

            # 6. ANÁLISIS PANÓPTICO
            metricas_panoptico = {}
            try:
                resultado_panoptico = self.processor.post_process_panoptic_segmentation(
                    outputs, target_sizes=[(h, w)]
                )[0]

                if 'segments_info' in resultado_panoptico:
                    segmentos = resultado_panoptico['segments_info']
                    categorias = [seg.get('category_id', -1) for seg in segmentos]

                    metricas_panoptico = {
                        'total_segmentos': len(segmentos),
                        'categorias_unicas': len(set(categorias)),
                        'areas_segmentos': [seg.get('area', 0) for seg in segmentos],
                        'distribucion_categorias': {str(k): categorias.count(k) for k in set(categorias)},
                        'segmentos_persona': sum(1 for seg in segmentos if seg.get('category_id') == 0)
                    }
            except:
                metricas_panoptico = {'error': 'Panóptico no disponible'}

            # 7. RECURSOS DESPUÉS DE INFERENCIA
            recursos_despues = self._obtener_recursos()
            tiempo_total = (time.time() - inicio) * 1000

            # 8. ANÁLISIS DE LA MEJOR DETECCIÓN (umbral 0.5 como referencia)
            mejor_resultado = resultados_por_umbral.get('umbral_0.5', {})

            # 9. CLASIFICACIÓN AUTOMÁTICA DE CONTEXTO
            contexto = self._clasificar_contexto_automatico(
                imagen_np,
                mejor_resultado.get('num_detecciones', 0),
                metricas_imagen
            )

            # 10. RESULTADO FINAL COMPLETO
            resultado_completo = {
                # Metadatos
                'timestamp': datetime.now().isoformat(),
                'id_procesamiento': f"{hash_img}_{int(time.time())}",

                # Datos de la imagen
                'imagen': metricas_imagen,

                # Rendimiento
                'rendimiento': {
                    'tiempo_inferencia_ms': tiempo_inferencia,
                    'tiempo_total_ms': tiempo_total,
                    'memoria_antes_mb': recursos_antes.get('memoria_usada_mb', 0),
                    'memoria_despues_mb': recursos_despues.get('memoria_usada_mb', 0),
                    'memoria_gpu_mb': recursos_despues.get('gpu_memoria_mb', 0),
                    'cpu_percent': recursos_despues.get('cpu_percent', 0)
                },

                # Resultados por umbral
                'segmentacion': resultados_por_umbral,

                # Análisis panóptico
                'panoptico': metricas_panoptico,

                # Contexto automático
                'contexto': contexto,

                # Modelo info
                'modelo': {
                    'nombre': self.config['model_name'],
                    'device': str(self.device),
                    'confianza_umbral_principal': 0.5
                }
            }

            del inputs, outputs, imagen_pil, imagen_np
            gc.collect()
            torch.cuda.empty_cache()

            return resultado_completo

        except Exception as e:
            return {
                'timestamp': datetime.now().isoformat(),
                'imagen': {'archivo': os.path.basename(ruta_imagen)},
                'error': str(e),
                'procesamiento_fallido': True
            }

    def _calcular_entropia(self, imagen_gray):
        """Calcula la entropía de Shannon de la imagen."""
        histogram, _ = np.histogram(imagen_gray, bins=256, range=(0, 256))
        histogram = histogram + 1e-7  # Evitar log(0)
        prob = histogram / np.sum(histogram)
        return float(-np.sum(prob * np.log2(prob)))

    def _calcular_densidad_bordes(self, imagen_gray):
        """Calcula densidad de bordes usando Canny."""
        edges = cv2.Canny(imagen_gray, 50, 150)
        return float(np.sum(edges > 0) / edges.size)

    def _metricas_coherencia(self, mascara):
        """Métricas de coherencia espacial de una máscara."""
        if np.sum(mascara) == 0:
            return {
                'area': 0,
                'componentes': 0,
                'compacidad': 0.0,
                'solidez': 0.0
            }

        # Componentes conectados
        labeled = measure.label(mascara)
        num_componentes = int(labeled.max())

        # Contornos para otras métricas
        contours, _ = cv2.findContours(mascara.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            contorno_principal = max(contours, key=cv2.contourArea)
            area = cv2.contourArea(contorno_principal)
            perimetro = cv2.arcLength(contorno_principal, True)

            # Convex hull
            hull = cv2.convexHull(contorno_principal)
            area_hull = cv2.contourArea(hull)

            # Métricas
            compacidad = (4 * np.pi * area) / (perimetro ** 2) if perimetro > 0 else 0
            solidez = area / area_hull if area_hull > 0 else 0

            return {
                'area': int(area),
                'componentes': num_componentes,
                'compacidad': float(compacidad),
                'solidez': float(solidez),
                'perimetro': float(perimetro)
            }

        return {
            'area': int(np.sum(mascara)),
            'componentes': num_componentes,
            'compacidad': 0.0,
            'solidez': 0.0
        }

    def _obtener_recursos(self):
        """Obtiene información de recursos del sistema."""
        try:
            mem = psutil.virtual_memory()
            recursos = {
                'memoria_usada_mb': mem.used / (1024*1024),
                'cpu_percent': psutil.cpu_percent()
            }

            if torch.cuda.is_available():
                recursos['gpu_memoria_mb'] = torch.cuda.memory_allocated() / (1024*1024)

            return recursos
        except:
            return {}

    def _clasificar_contexto_automatico(self, imagen, num_personas, metricas_img):
        """Clasificación automática de contexto basada en métricas."""

        # Reglas simples de clasificación
        if num_personas == 0:
            categoria = 'sin_personas'
        elif num_personas == 1:
            if metricas_img['densidad_bordes'] < 0.1:
                categoria = 'retrato_simple'
            else:
                categoria = 'retrato_complejo'
        else:
            categoria = 'multiples_personas'

        # Análisis de complejidad
        complejidad = (
            metricas_img['densidad_bordes'] * 0.4 +
            min(metricas_img['varianza_laplacian'] / 1000, 1.0) * 0.3 +
            (metricas_img['contraste_local_std'] / 100) * 0.3
        )

        return {
            'categoria': categoria,
            'complejidad_score': float(min(complejidad, 1.0)),
            'num_personas_detectadas': num_personas,
            'iluminacion': 'baja' if metricas_img['brillo_promedio'] < 80 else
                         'alta' if metricas_img['brillo_promedio'] > 180 else 'normal'
        }

    def evaluar_dataset(self, rutas_imagenes):
        """Evalúa un conjunto de imágenes completo."""
        print(f"Iniciando evaluación de {len(rutas_imagenes)} imágenes")

        self.resultados = []
        exitosas = 0

        for i, ruta in enumerate(rutas_imagenes):
            print(f"[{i+1:4d}/{len(rutas_imagenes)}] {os.path.basename(ruta)}")

            resultado = self.procesar_imagen(ruta)
            self.resultados.append(resultado)

            if not resultado.get('procesamiento_fallido', False):
                exitosas += 1
                num_personas = resultado.get('segmentacion', {}).get('umbral_0.5', {}).get('num_detecciones', 0)
                tiempo = resultado.get('rendimiento', {}).get('tiempo_total_ms', 0)
                print(f"  {num_personas} personas, {tiempo:.1f}ms")
            else:
                print(f"  Error: {resultado.get('error', 'unknown')}")

            # Limpieza periódica de memoria
            if (i + 1) % 10 == 0:
                torch.cuda.empty_cache()

        print(f"\nResumen: {exitosas}/{len(rutas_imagenes)} exitosas")
        return self.resultados

    def guardar_resultados(self, nombre_archivo=None):
        """Guarda todos los resultados en formato JSON."""
        if not nombre_archivo:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            nombre_archivo = f"evaluacion_mask2former_{timestamp}.json"

        ruta_archivo = self.output_dir / 'datos_completos' / nombre_archivo

        # Preparar datos para JSON
        datos_exportacion = {
            'resumen': {
                'total_imagenes': len(self.resultados),
                'exitosas': sum(1 for r in self.resultados if not r.get('procesamiento_fallido', False)),
                'timestamp': datetime.now().isoformat(),
                'modelo': self.config['model_name']
            },
            'configuracion': self.config,
            'resultados': self.resultados
        }

        with open(ruta_archivo, 'w', encoding='utf-8') as f:
            json.dump(datos_exportacion, f, indent=2, ensure_ascii=False)

        print(f"Resultados guardados en: {ruta_archivo}")
        print(f"Tamaño del archivo: {os.path.getsize(ruta_archivo) / (1024*1024):.2f} MB")

        return str(ruta_archivo)

In [39]:
# ==========================================
# 2. Función para cargar dataset desde carpeta
# ==========================================
def cargar_dataset(ruta_dataset, extensiones=(".jpg", ".png", ".jpeg")):
    ruta = Path(ruta_dataset)
    imagenes = [str(p) for p in ruta.glob("**/*") if p.suffix.lower() in extensiones]
    print(f"Dataset cargado: {len(imagenes)} imágenes encontradas.")
    return imagenes

In [40]:
# ==========================================
# 3. Ejecutor por lotes con limpieza de GPU
# ==========================================
def procesar_por_lotes(evaluador, imagenes, tam_lote=5):
    resultados_totales = []
    for i in tqdm(range(0, len(imagenes), tam_lote), desc="Procesando lotes"):
        lote = imagenes[i:i+tam_lote]
        resultados_lote = evaluador.evaluar_dataset(lote)
        resultados_totales.extend(resultados_lote)
        gc.collect()
        torch.cuda.empty_cache()
    return resultados_totales

In [41]:
def generar_resumen(resultados):
    resumen = {}
    for r in resultados:
        for clave, valor in r.items():
            if isinstance(valor, (int, float)):
                resumen.setdefault(clave, []).append(valor)
    return {k: sum(v)/len(v) for k, v in resumen.items() if v}

In [42]:
# ==========================================
# 4. Ejecución sobre múltiples modelos
# ==========================================
def ejecutar_multi_modelo(modelos, dataset_path, drive_output_path, tam_lote=1):
    imagenes = cargar_dataset(dataset_path)

    resumen_global = {}

    for modelo in modelos:
        print(f"\n=========================")
        print(f"Ejecutando modelo: {modelo}")
        print(f"=========================")

        config = {
            'model_name': modelo,
            'device': "cuda" if torch.cuda.is_available() else "cpu",
            'output_dir': drive_output_path
        }

        evaluador = RecolectorMetricasCompletas(config)
        resultados = procesar_por_lotes(evaluador, imagenes, tam_lote)

        # Guardar resultados por modelo
        nombre_archivo = f"resultados_{modelo.replace('/', '_')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        ruta_archivo = Path(drive_output_path) / nombre_archivo
        with open(ruta_archivo, 'w', encoding='utf-8') as f:
            json.dump(resultados, f, indent=2, ensure_ascii=False)
        print(f"Resultados guardados en: {ruta_archivo}")

        resumen_global[modelo] = generar_resumen(resultados)

    # Guardar resumen global
    archivo_resumen = Path(drive_output_path) / f"resumen_global_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(archivo_resumen, 'w', encoding='utf-8') as f:
        json.dump(resumen_global, f, indent=2, ensure_ascii=False)
    print(f"\nResumen global guardado en: {archivo_resumen}")

In [43]:
# ==========================================
# 5. Configuración y ejecución principal
# ==========================================
if __name__ == "__main__":
    ejecutar_multi_modelo(MODELOS, DATASET_PATH, OUTPUT_PATH, tam_lote=5)

Dataset cargado: 3 imágenes encontradas.

Ejecutando modelo: facebook/mask2former-swin-large-coco-instance
Modelo cargado en cuda


Procesando lotes:   0%|          | 0/1 [00:00<?, ?it/s]

Iniciando evaluación de 3 imágenes
[   1/3] _DSC0320.jpg
  Error: 'RecolectorMetricasCompletas' object has no attribute 'read'
[   2/3] _DSC0160.jpg
  Error: 'RecolectorMetricasCompletas' object has no attribute 'read'
[   3/3] _DSC1017.jpg
  Error: 'RecolectorMetricasCompletas' object has no attribute 'read'

Resumen: 0/3 exitosas


Procesando lotes: 100%|██████████| 1/1 [00:00<00:00,  2.86it/s]


Resultados guardados en: /content/drive/MyDrive/TFM/mask2former/resultados/resultados_facebook_mask2former-swin-large-coco-instance_20250826_205928.json

Ejecutando modelo: facebook/maskformer-swin-base-coco


ValueError: Unrecognized configuration class <class 'transformers.models.maskformer.configuration_maskformer.MaskFormerConfig'> for this kind of AutoModel: AutoModelForSemanticSegmentation.
Model type should be one of BeitConfig, Data2VecVisionConfig, DPTConfig, MobileNetV2Config, MobileViTConfig, MobileViTV2Config, SegformerConfig, UperNetConfig.